In [1]:
using Flux
using Flux: Chain, Dense, relu, mse, train!, params, ADAM
using Random
using Gogeta
using Plots
using Gurobi
using JuMP
using LaTeXStrings
using CSV
using DataFrames

In [2]:
function Psplits(X::AbstractVector, P::Integer)
    #l%P sub-arrays of size (1//P)+1 and the rest 1//P
    #returns the sorted indices
    parts = Vector{Vector{Int64}}(undef, P)

    l = length(X)
    n = l % P

    s2 = (l÷P)

    if n > 0
        s1 = (l÷P)+1
        for k in 1:n
            parts[k] = X[1+(k-1)*s1:(k-1)*s1+s1]
        end
    else
        s1 = 0
    end
    
    for k in n+1:P
        parts[k] = X[1+n*s1+(k-1-n)*s2:n*s1+(k-1-n)*s2+s2]
    end
    
    return parts
end

Psplits (generic function with 1 method)

In [3]:
function NN_formulate_Psplit!(jump_model::JuMP.Model, NN_model::Flux.Chain, P, init_U, init_L; silent=true)

    oldstdout = stdout
    if silent redirect_stdout(devnull) end
    println("Creating JuMP model...")
    empty!(jump_model)

    K = length(NN_model); # number of layers (input layer not included)
    W = deepcopy([Flux.params(NN_model)[2*k-1] for k in 1:K]);
    b = deepcopy([Flux.params(NN_model)[2*k] for k in 1:K]);


    @assert all([NN_model[i].σ == relu for i in 1:K-1]) "Neural network must use the relu activation function."
    @assert NN_model[K].σ == identity "Neural network must use the identity function for the output layer."
    @assert P > 0 "The number of splits must be more than 0."

    input_length = Int((length(W[1]) / length(b[1])))
    neuron_count = [length(b[k]) for k in eachindex(b)]
    neurons(layer) = layer == 0 ? [i for i in 1:input_length] : [i for i in 1:neuron_count[layer]]

    @variable(jump_model, x[layer = 0:K, neurons(layer)]);
    @variable(jump_model, sigma[layer = 1:K-1, neurons(layer)]);
    @variable(jump_model, z_b[layer = 1:K-1, neurons(layer), p=1:P]);

    @constraint(jump_model, [j = 1:input_length], x[0, j] <= init_U[j])
    @constraint(jump_model, [j = 1:input_length], x[0, j] >= init_L[j])

    bounds_U = Vector{Vector}(undef, K)
    bounds_L = Vector{Vector}(undef, K)

    UB_α = Vector{Vector{Vector}}(undef, K-1)
    LB_α = Vector{Vector{Vector}}(undef, K-1)

    [UB_α[layer] = Vector{Vector}(undef, neuron_count[layer]) for layer in 1:K-1]
    [LB_α[layer] = Vector{Vector}(undef, neuron_count[layer]) for layer in 1:K-1]

    for layer in 1:K # hidden layers and output layer

        println("\nLAYER $layer")

        if layer == 1

            bounds_U[layer] = [sum(max(W[layer][neuron, previous] * init_U[previous], W[layer][neuron, previous] * init_L[previous]) for previous in neurons(layer-1)) + b[layer][neuron] for neuron in neurons(layer)]
            bounds_L[layer] = [sum(min(W[layer][neuron, previous] * init_U[previous], W[layer][neuron, previous] * init_L[previous]) for previous in neurons(layer-1)) + b[layer][neuron] for neuron in neurons(layer)]
        else
            
            bounds_U[layer] = [sum(bounds_U[layer-1][previous]*max(0, W[layer][neuron, previous]) + bounds_L[layer-1][previous]*min(0, W[layer][neuron, previous]) for previous in neurons(layer-1)) + b[layer][neuron] for neuron in neurons(layer)]
            bounds_L[layer] = [sum(bounds_L[layer-1][previous]*max(0, W[layer][neuron, previous]) + bounds_U[layer-1][previous]*min(0, W[layer][neuron, previous]) for previous in neurons(layer-1)) + b[layer][neuron] for neuron in neurons(layer)]
        end
        # output bounds calculated but no more constraints added
        if layer == K
            break
        end

        [UB_α[layer][neuron] = Vector(undef, P) for neuron in 1:neuron_count[layer]]
        [LB_α[layer][neuron] = Vector(undef, P) for neuron in 1:neuron_count[layer]]

        @constraint(jump_model, [neuron in neurons(layer)], x[layer, neuron] <= max(0, bounds_U[layer][neuron]))
        @constraint(jump_model, [neuron in neurons(layer)], x[layer, neuron] >= 0)

        for neuron in neurons(layer)

            split_indices = Psplits(sortperm(W[layer][neuron, :]), P)
            set_binary(sigma[layer, neuron])
            
            @constraint(jump_model, sum(sum(W[layer][neuron, i]*x[layer-1, i] for i in split_indices[p])-z_b[layer, neuron, p] for p in 1:P) + sigma[layer, neuron]*b[layer][neuron]<=0)
            @constraint(jump_model, sum(z_b[layer, neuron, p] for p in 1:P) + (1-sigma[layer,neuron])*b[layer][neuron]>=0)
            @constraint(jump_model, x[layer, neuron] == sum(z_b[layer, neuron, p] for p in 1:P) + (1-sigma[layer,neuron])*b[layer][neuron]) 

            for p in 1:P
            
                if layer == 1
                    UB_α[layer][neuron][p] = sum(max(W[layer][neuron, previous] * init_U[previous], W[layer][neuron, previous] * init_L[previous]) for previous in split_indices[p])
                    LB_α[layer][neuron][p] = sum(min(W[layer][neuron, previous] * init_U[previous], W[layer][neuron, previous] * init_L[previous]) for previous in split_indices[p])
                else
                    UB_α[layer][neuron][p] = sum(max(W[layer][neuron, previous] * max(0, bounds_U[layer-1][previous]), W[layer][neuron, previous] * max(0, bounds_L[layer-1][previous])) for previous in split_indices[p])
                    LB_α[layer][neuron][p] = sum(min(W[layer][neuron, previous] * max(0, bounds_U[layer-1][previous]), W[layer][neuron, previous] * max(0, bounds_L[layer-1][previous])) for previous in split_indices[p])
                end
                

                @constraint(jump_model, sigma[layer, neuron]*LB_α[layer][neuron][p]<=sum(W[layer][neuron, i]*x[layer-1, i] for i in split_indices[p])-z_b[layer, neuron, p])
                @constraint(jump_model, sum(W[layer][neuron, i]*x[layer-1, i] for i in split_indices[p])-z_b[layer, neuron, p]<=sigma[layer, neuron]*UB_α[layer][neuron][p]) 
                @constraint(jump_model, (1-sigma[layer, neuron])*LB_α[layer][neuron][p]<=z_b[layer, neuron, p])
                @constraint(jump_model, z_b[layer, neuron, p]<=(1-sigma[layer, neuron])*UB_α[layer][neuron][p])

            end
        end
    end

    # output layer
    @constraint(jump_model, [neuron in neurons(K)], x[K, neuron] <= max(0, bounds_U[K][neuron]))
    @constraint(jump_model, [neuron in neurons(K)], x[K, neuron] >= min(0, bounds_L[K][neuron]))
    @constraint(jump_model, [neuron in neurons(K)], x[K, neuron] == b[K][neuron] + sum(W[K][neuron, i] * x[K-1, i] for i in neurons(K-1)))

    #A dummy objective
    @objective(jump_model, Max, 1);
    redirect_stdout(oldstdout)

    return bounds_U, bounds_L
end

NN_formulate_Psplit! (generic function with 1 method)

In [6]:
begin
    Random.seed!(1234);

    NN_model = Chain(
        Dense(4 => 8, relu),
        Dense(8 => 16, relu),
        Dense(16 => 1)
        
    )
end

init_U = [1, 1, 1, 1];
init_L = [0, 0, 0, 0];

In [7]:
jump_model = Model(Gurobi.Optimizer);
set_silent(jump_model) #disables output of the optimazer
set_attribute(jump_model, "TimeLimit", 10) #sets the time limit to be 10 seconds

P = 2
@time bounds_U, bounds_L = NN_formulate_Psplit!(jump_model, NN_model, P, init_U, init_L)
jump_model

Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-20
  0.228972 seconds (418.50 k allocations: 28.285 MiB, 4.84% gc time, 99.12% compilation time)


A JuMP Model
Maximization problem with:
Variables: 101
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 25 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 53 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 245 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 24 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: sigma, x, z_b

In [8]:
output_neuron = jump_model[:x][maximum(keys(jump_model[:x].data))]
@objective(jump_model, Max, output_neuron);

@time optimize!(jump_model)
println("The model found next solution:\n", value.(jump_model[:x][0, :]))
println("With objective function: ", objective_value(jump_model) )
solution = Float32.([i for i in value.(jump_model[:x][0, :])])
println("The output of the NN for solution given by jump model: ", NN_model(solution)[1])

  0.537365 seconds (1.69 M allocations: 112.613 MiB, 6.10% gc time, 97.39% compilation time: 4% of which was recompilation)
The model found next solution:
  [1]  =  1.0
  [2]  =  1.0
  [3]  =  0.0
  [4]  =  0.0
With objective function: 0.40986967350298237
The output of the NN for solution given by jump model: 0.40986964


In [10]:
forward_pass!(jump_model, [1, 1, 1, 1]), NN_model([1,1,1,1])

([0.10855757781568458], Float32[0.10855755])